In [ ]:
#%pip install openpyxl pandas

In [1]:
from openpyxl import Workbook, load_workbook
from win32com import client
import shutil
import pandas as pd
import os
from pywintypes import com_error
from pathlib import Path
import shutil

def eds_file(dfs, col_rows, jz, eds, dir_save: Path, dir_base: Path):
    """
    Genera un archivo Excel para un EDS específico copiando un molde
    y completando datos desde los DataFrames.
    """

    # Archivo molde y destino
    dir_archivo_molde = dir_base / "Carga Inicial" / "NNCC" / "Formato POA JZ - Negocios Complementarios.xlsx"
    print(dir_save)
    shutil.copy2(dir_archivo_molde, dir_save)

    # Cargar Excel
    workbook = load_workbook(dir_save)
    worksheet = workbook.active

    # Escribir código EDS en celda fija
    worksheet.cell(row=10, column=1, value=eds)

    # Rellenar datos desde los DataFrames
    for df_name, df in dfs.items():
        print(f"Procesando {df_name}...")
        print(df.head())
        col_start, row_start = col_rows[df_name]

        df_filtered = df.loc[df.CodEs == eds]

        if df_filtered.empty:
            print(f"{df_name}: Vacío o sin DF")
            continue

        # Tomar solo la primera fila a partir de la 4ta columna (índice 3)
        row_values = df_filtered.iloc[0, 3:].values

        for col_offset, valor in enumerate(row_values):
            worksheet.cell(
                row=row_start, 
                column=col_start + col_offset, 
                value=valor
            )

    # Guardar cambios
    workbook.save(dir_save)


### Carga Archivos versión PRE CARGA INICIAL (Pre Programa POA en combustible)

In [2]:
anno_poa = 2026
# Directorio base común
base_dir = Path(
    rf"G:\Unidades compartidas\2_Planificación Comercial - EDS\4. POA\POA Gestión\POA {anno_poa}\POA Comercial - Carga Inicial"
)

# Archivos
files = {
    "maestro_eds": base_dir / "Maestro EDS.xlsx",
    "punto": base_dir / "Carga Inicial" / "NNCC" / f"POA {anno_poa} - Punto.xlsx",
    "lavados": base_dir / "Carga Inicial" / "NNCC" / f"POA {anno_poa} - Lavado.xlsx",
    "lub": base_dir / "Carga Inicial" / "NNCC" / f"POA {anno_poa} - LUB.xlsx",
    "vta_isla": base_dir / "Carga Inicial" / "NNCC" / f"POA {anno_poa} - Venta en isla.xlsx",
}
# zonas = [209, 210, 211, 212, 213, 214, 215, 216, 217, 
#         218, 219, 220, 221, 230, 231, 232, 233, 234, 
#         235, 240, 241, 242, 243, 244, 245, 246, 247, 
#         248, 249]
zonas = [214]


maestro_eds = pd.read_excel(
    files["maestro_eds"], usecols=[0, 5], sheet_name="Consolidado"
)
estacionales = pd.read_excel(
    files["maestro_eds"],
    usecols=[0, 12],
    names=["CodEs", "JZ"],
    sheet_name="Estacionales",
)
# ficticias = pd.read_excel(files["maestro_eds"], usecols=[0], names=['CodEs'], sheet_name='Ficticias')
# ficticias.insert(1, 'Estado', 'Ficticia')
maestro_eds = pd.concat([maestro_eds, estacionales])
maestro_eds["CodEs"] = maestro_eds["CodEs"].fillna(0).astype(int)
maestro_eds["JZ"] = maestro_eds["JZ"].fillna(0).astype(int)

# Archivos Punto
base_punto_cig = pd.read_excel(files["punto"], sheet_name="Cigarros").fillna(0)
base_punto_vnop = pd.read_excel(files["punto"], sheet_name="Otros productos").fillna(0)

# Archivos Lavado
base_lavados_auto = pd.read_excel(files["lavados"], sheet_name="Autoservicio").fillna(0)
base_lavados_arco = pd.read_excel(files["lavados"], sheet_name="Arcos").fillna(0)
base_lavados_tunel = pd.read_excel(files["lavados"], sheet_name="Tunel").fillna(0)
base_lavados_aspirado = pd.read_excel(files["lavados"], sheet_name="Aspirado").fillna(0)
base_lavados_interior = pd.read_excel(files["lavados"], sheet_name="Interior").fillna(0)

# Archivos LUB
base_lub_cambios = pd.read_excel(files["lub"], sheet_name="Cambios Lub").fillna(0)
base_lub_filtro_aire = pd.read_excel(files["lub"], sheet_name="Filtros Lub").fillna(0)

# Archivos Venta en Isla
base_isla_agua = pd.read_excel(files["vta_isla"], sheet_name="Aguas VF").fillna(0)
base_isla_lub = pd.read_excel(files["vta_isla"], sheet_name="Lubricantes VF").fillna(0)
base_isla_bidon = pd.read_excel(files["vta_isla"], sheet_name="Bidones VF").fillna(0)
base_isla_bluemax = pd.read_excel(files["vta_isla"], sheet_name="Bluemax VF").fillna(0)

In [3]:
dfs = {
    "Punto - CIG": base_punto_cig,
    "Punto - VNOP": base_punto_vnop,
    "Lavados": base_lavados_auto,
    "Arco": base_lavados_arco,
    "Tunel": base_lavados_tunel,
    "Aspirado": base_lavados_aspirado,
    "Interior": base_lavados_interior,
    "LUB Cambios": base_lub_cambios,
    "LUB Filtro Aire": base_lub_filtro_aire,
    "Isla - Agua": base_isla_agua,
    "Isla- LUB": base_isla_lub,
    "Isla - Bidón": base_isla_bidon,
    "Isla - Bluemax": base_isla_bluemax,
}


col_rows = {
    "Punto - CIG": [4, 10],
    "Punto - VNOP": [4, 11],
    "Lavados": [4, 13],
    "Arco": [4, 14],
    "Tunel": [4, 15],
    "Aspirado": [4, 16],
    "Interior": [4, 17],
    "LUB Cambios": [4, 19],
    "LUB Filtro Aire": [4, 20],
    "Isla - Agua": [4, 22],
    "Isla- LUB": [4, 23],
    "Isla - Bidón": [4, 24],
    "Isla - Bluemax": [4, 25],
}

In [4]:
# Generación archivos

# Directorio base
output_dir = base_dir / "Carga Inicial" / "Archivos por Zona Respaldo"

# Crear las carpetas por zona si no existen
for zona in zonas:
    directory = output_dir / str(zona)
    directory.mkdir(parents=True, exist_ok=True)

cont = 1
for jz in maestro_eds.JZ.unique():
    for eds in maestro_eds.loc[maestro_eds.JZ == jz, "CodEs"]:
        
        if(eds != 60293):
            continue
        
        output_file = output_dir / str(jz) / f"{eds} - NNCC.xlsx"
        eds_file(dfs, col_rows, jz, eds, output_file, dir_base=base_dir)
        print(cont, eds, jz)
        cont += 1


G:\Unidades compartidas\2_Planificación Comercial - EDS\4. POA\POA Gestión\POA 2026\POA Comercial - Carga Inicial\Carga Inicial\Archivos por Zona Respaldo\214\60293 - NNCC.xlsx
Procesando Punto - CIG...
   CodEs Producto               SubProducto       2024-1       2024-2  \
0  10004    Punto  Venta Neta Cigarros ($M)  37557.20443  33865.51286   
1  10006    Punto  Venta Neta Cigarros ($M)  51671.87222  43973.17747   
2  10021    Punto  Venta Neta Cigarros ($M)  18070.77821  15167.48212   
3  10024    Punto  Venta Neta Cigarros ($M)  20390.37186  18415.94931   
4  10027    Punto  Venta Neta Cigarros ($M)      0.00000      0.00000   

        2024-3       2024-4       2024-5       2024-6       2024-7  ...  \
0  36243.32753  34035.21877  33687.77671  32089.64301  33417.51335  ...   
1  45334.49481  41996.49314  41765.30791  41443.69120  41727.58242  ...   
2  15611.75810  15059.78048  14869.94945  14745.27084  14760.66299  ...   
3  20926.60652  18133.03849  18791.62928  17650.47900  186

In [ ]:
# Directorio base común
output_dir = base_dir / "Carga Inicial" / "Archivos por Zona Respaldo"

# Generación archivo para un EDS específico
output_file = output_dir / "220" / "10056 - NNCC.xlsx"
print(output_file)
eds_file(dfs, col_rows, 220, 10056, output_file, dir_base=base_dir)